# Create HINO Vectors
<b>Author</b>: Ian Coleman <br/>
<b>Function</b>: Create vectors for Chems/Diseases on the Human Interaction Network Ontology. I'll get the Chem/Dis - pathways associations from CTD. These are inferred (I believe from the gene associations...?)

In [150]:
import pandas as pd
import numpy as np
import scipy as sp
import subprocess
import math
import re

### First we need to get the HINO IDs from the REACT and KEGG IDs that CTD uses

In [151]:
# The HINO csv has some links to REACT IDs, let's start there 


#it has lots of columns, let's limit these
# colnames= ['']
# Class ID,Preferred Label,Synonyms,Definitions,Obsolete,CUI,Semantic Types,Parents,alternative term,bearer of,before,before or simultaneous with,BFO CLIF specification label,BFO OWL specification label,causal relation between processes,causally downstream of,causally downstream of or within,causally related to,causally upstream of,causally upstream of or within,"causally upstream of, negative effect","causally upstream of, positive effect",Contributor,controls,Coverage,Creator,curator note,database_cross_reference,Date,definition,definition source,depends on,Description,directly activates,directly inhibits,directly negatively regulates,directly positively regulates,directly provides input for,directly regulates,during which ends,during which starts,editor note,editor preferred term,elucidation,encompasses,ends,ends after,ends during,ends with,example of literature mining keyword usage,example of usage,expand expression to,Format,functionally related to,happens during,has activator,has associated axiom(fol),has associated axiom(nl),has component,has component activity,has component process,has curation status,has direct input,has effector activity,has input,has interaction type,has keyword dependency pattern,has literature mining keywords,has output,has part,has participant,has role,has_alternative_id,has_broad_synonym,has_controller,has_exact_synonym,has_narrow_synonym,has_next_step,has_obo_namespace,has_rank,has_related_synonym,http://data.bioontology.org/metadata/obo/part_of,http://data.bioontology.org/metadata/prefixIRI,http://data.bioontology.org/metadata/treeView,http://purl.obolibrary.org/obo/hasExactSynonym,http://purl.obolibrary.org/obo/hasNarrowSynonym,http://purl.obolibrary.org/obo/hasOBONamespace,http://purl.obolibrary.org/obo/id,http://purl.obolibrary.org/obo/mi#PSI-MI_slim,http://www.biopax.org/release/biopax-level3.owl#controlled,http://www.biopax.org/release/biopax-level3.owl#controlType,http://www.biopax.org/release/biopax-level3.owl#name,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.biopax.org/release/biopax-level3.owl#product,http://www.geneontology.org/formats/oboInOwl#created_by,http://www.geneontology.org/formats/oboInOwl#creation_date,http://www.geneontology.org/formats/oboInOwl#id,http://www.obofoundry.org/ro/ro.owl#has_part,http://www.obofoundry.org/ro/ro.owl#located_in,http://www.w3.org/2000/01/rdf-schema#comment,http://www.w3.org/2000/01/rdf-schema#isDefinedBy,http://www.w3.org/2004/02/skos/core#notation,http://xmlns.com/foaf/0.1/homepage,http://xmlns.com/foaf/0.1/page,immediately causally downstream of,immediately causally upstream of,immediately preceded by,immediately precedes,imported from,in_subset,indirectly activates,indirectly inhibits,inheres in,inheres in part of,input of,is direct form of,is indirect form of,label,lacks part,Language,logical macro assertion,logical macro assertion on a property,logical macro assertion on an annotation property,logical macro assertion on an object property,mereotopologically related to,negatively regulated by,negatively regulates,OBO foundry unique label,obsolete has direct output,obsolete preceded by,output of,overlaps,part of,participates in,positively regulated by,positively regulates,preceded by,precedes,Publisher,realized in,realizes,regulated by,regulates,related via dependence to,Relation,Resource Identifier,Resource Type,Rights Management,simultaneous with,Source,starts,starts before,starts during,starts with,Subject and Keywords,temporal interpretation,temporally related to,term editor,Title,transitively provides input for
hino = pd.read_csv('/home/ian/Downloads/HINO.csv').iloc[:,0:4]#, usecols=['uri', 'label', 'synonyms', 'defs'])

/home/ian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,12,13,18,25,41,42,43,48,49,50,56,57,61,65,66,70,71,73,76,79,81,83,84,85,86,87,94,95,100,126,135,139,142) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [152]:
# Use regex to get the reactome ID from Definitions
r = re.compile('Reactome DB_ID: [0-9]*')
vmatch = np.vectorize(lambda x:bool(r.match(x)))

# func to extract the reactome bit
def get_react_id(x, r):
    if x is not None:
        results = r.search(x)
        if results is None: return None
        else: return results[0].replace('Reactome DB_ID: ', 'REACT:R-HSA-')
    else:
        return None

# Convert to str and extract
hino['Definitions'] = hino.Definitions.astype(str)
hino['Definitions'] = hino.Definitions.map(lambda x: get_react_id(x,r))

In [153]:
hino.sample(13)

,Class ID,Preferred Label,Synonyms,Definitions
16701,http://purl.obolibrary.org/obo/UniProt_P66791,UniProt_P66791,NaN,None
32526,http://purl.obolibrary.org/obo/HINO_0021208,PathwayStep5381,NaN,None
16052,http://purl.obolibrary.org/obo/CHEBI_64039,(2S)-pristanoyl-CoA,"3'-phosphoadenosine 5'-{3-[(3R)-3-hydroxy-2,2-...",None
12306,http://purl.obolibrary.org/obo/UniProt_P45844-3,UniProt_P45844-3,NaN,None
17171,http://purl.obolibrary.org/obo/HINO_0027156,DNA containing an MPG-bound apurinic/apyrimidi...,NaN,REACT:R-HSA-110207
15023,http://purl.obolibrary.org/obo/HINO_0026083,G beta:gamma signalling through PLC beta,NaN,None
35195,http://purl.obolibrary.org/obo/HINO_0006775,Gycosylated NA Tetramer:Lipid Raft,NaN,REACT:R-HSA-195735
17523,http://purl.obolibrary.org/obo/HINO_0006562,PGH2 is reduced to PGF2a by FAM213B,NaN,None
24418,http://purl.obolibrary.org/obo/HINO_0019310,Tropomyosin,NaN,REACT:R-HSA-390545
27083,http://purl.obolibrary.org/obo/HINO_0026035,PathwayStep3550,NaN,None


In [154]:
hino = hino.dropna(subset=['Definitions'])
print(len(hino))

9119


In [155]:
hino.sample(6)

,Class ID,Preferred Label,Synonyms,Definitions
5508,http://purl.obolibrary.org/obo/HINO_0009569,GlcNAc-GlcA-GlcNAc,NaN,REACT:R-HSA-2162223
13508,http://purl.obolibrary.org/obo/HINO_0013351,Ubiquitinated AUF1 Tetramer,NaN,REACT:R-HSA-450417
18984,http://purl.obolibrary.org/obo/HINO_0023030,SREBP1A/1C:ACACB gene,NaN,REACT:R-HSA-2426125
6350,http://purl.obolibrary.org/obo/HINO_0010376,phospho-p27/p21,NaN,REACT:R-HSA-187849
25327,http://purl.obolibrary.org/obo/HINO_0005334,Integrin alpha IIb beta 3:p(Y530)-SRC:CSK:Tali...,NaN,REACT:R-HSA-443900
32077,http://purl.obolibrary.org/obo/HINO_0012005,ADAM:Zn2+,NaN,REACT:R-HSA-179842


In [156]:
# Create map of REACT ID to URI

# Make the maps from this
react_map = dict(zip(hino.Definitions, hino['Class ID']))

In [157]:
# read in chem-path and dis-path
# create associations file
# create entities file
# run opa2vec
# Switch to opa-nn and import the vectors

In [158]:
# Read in Chem path
df_cpath = pd.read_csv('../ctd-to-nt/csvs/CTD_chem_pathways_enriched.csv.gz', skiprows=27)
df_cpath = df_cpath.drop(0)

# Read in dis path
df_dp = pd.read_csv('../ctd-to-nt/csvs/CTD_diseases_pathways.csv', skiprows=27)
df_dp = df_dp.drop(0)

In [159]:
# Create association file
df_cpath.sample(3)

,# ChemicalName,ChemicalID,CasRN,PathwayName,PathwayID,PValue,CorrectedPValue,TargetMatchQty,TargetTotalQty,BackgroundMatchQty,BackgroundTotalQty
18193,2-(2-amino-3-methoxyphenyl)-4H-1-benzopyran-4-one,C093973,NaN,Pancreatic secretion,KEGG:hsa04972,1.330000e-09,0.000002,11.0,391.0,96.0,43067.0
201894,Benzamides,D001549,NaN,Arginine biosynthesis,KEGG:hsa00220,8.960000e-09,0.000020,13.0,4297.0,21.0,43067.0
70017,"7-(benzylamino)-1,3,4,8-tetrahydropyrrolo(4,3,...",C553817,NaN,Thyroid hormone signaling pathway,KEGG:hsa04919,1.420000e-07,0.000100,6.0,87.0,116.0,43067.0


In [160]:
# ChemicalID PathwayID URI
df_cpath['URI'] = df_cpath.PathwayID.map(lambda x: react_map.get(x))
# # Apply the maps to df1
# df1['DOID'] = df1.DiseaseID.map(lambda x: dis_map.get(x))
# df1['CID'] = df1.ChemicalID.map(lambda x: chem_map.get(x))

In [162]:
df_cpath.URI.nunique()

0

In [146]:
tester = df_cpath.PathwayID.unique()
tester = [x for x in tester if 'REACT' in x]
sorted(tester)

['REACT:R-HSA-1059683',
 'REACT:R-HSA-109581',
 'REACT:R-HSA-109582',
 'REACT:R-HSA-109606',
 'REACT:R-HSA-109688',
 'REACT:R-HSA-109703',
 'REACT:R-HSA-109704',
 'REACT:R-HSA-110056',
 'REACT:R-HSA-110312',
 'REACT:R-HSA-110313',
 'REACT:R-HSA-110314',
 'REACT:R-HSA-110320',
 'REACT:R-HSA-110328',
 'REACT:R-HSA-110329',
 'REACT:R-HSA-110330',
 'REACT:R-HSA-110331',
 'REACT:R-HSA-110357',
 'REACT:R-HSA-110362',
 'REACT:R-HSA-110373',
 'REACT:R-HSA-110381',
 'REACT:R-HSA-111367',
 'REACT:R-HSA-111446',
 'REACT:R-HSA-111447',
 'REACT:R-HSA-111448',
 'REACT:R-HSA-111452',
 'REACT:R-HSA-111453',
 'REACT:R-HSA-111457',
 'REACT:R-HSA-111458',
 'REACT:R-HSA-111459',
 'REACT:R-HSA-111461',
 'REACT:R-HSA-111463',
 'REACT:R-HSA-111464',
 'REACT:R-HSA-111465',
 'REACT:R-HSA-111469',
 'REACT:R-HSA-111471',
 'REACT:R-HSA-111885',
 'REACT:R-HSA-111931',
 'REACT:R-HSA-111932',
 'REACT:R-HSA-111933',
 'REACT:R-HSA-111957',
 'REACT:R-HSA-111995',
 'REACT:R-HSA-111996',
 'REACT:R-HSA-111997',
 'REACT:R-

In [149]:
tester2 = react_map.keys()
print(len(tester2))
sorted(tester2)

9119


['REACT:R-HSA-1006146',
 'REACT:R-HSA-1006173',
 'REACT:R-HSA-1008206',
 'REACT:R-HSA-1008217',
 'REACT:R-HSA-1008228',
 'REACT:R-HSA-1008229',
 'REACT:R-HSA-1008234',
 'REACT:R-HSA-1008247',
 'REACT:R-HSA-1008252',
 'REACT:R-HSA-1011577',
 'REACT:R-HSA-1011595',
 'REACT:R-HSA-1011605',
 'REACT:R-HSA-1012969',
 'REACT:R-HSA-1012978',
 'REACT:R-HSA-1012988',
 'REACT:R-HSA-1013010',
 'REACT:R-HSA-1013011',
 'REACT:R-HSA-1013017',
 'REACT:R-HSA-1013018',
 'REACT:R-HSA-1013019',
 'REACT:R-HSA-1013836',
 'REACT:R-HSA-1013858',
 'REACT:R-HSA-1013880',
 'REACT:R-HSA-1014260',
 'REACT:R-HSA-1015692',
 'REACT:R-HSA-1015693',
 'REACT:R-HSA-1015695',
 'REACT:R-HSA-1015696',
 'REACT:R-HSA-1015697',
 'REACT:R-HSA-1015817',
 'REACT:R-HSA-1015827',
 'REACT:R-HSA-1015869',
 'REACT:R-HSA-1017213',
 'REACT:R-HSA-1017219',
 'REACT:R-HSA-1018384',
 'REACT:R-HSA-1018386',
 'REACT:R-HSA-1022113',
 'REACT:R-HSA-1022114',
 'REACT:R-HSA-1022115',
 'REACT:R-HSA-1022117',
 'REACT:R-HSA-1022134',
 'REACT:R-HSA-10

In [165]:
'REACT:R-HSA-9033520' in tester2

False

In [139]:
len(react_map)

9119

In [ ]:
# Get uniprot ids for chem dis (already have somewhere)
# Create map of uniprot to HINO as above
# Game set match